The data for this notebook comes from our propriety fingerspelling image collection, which per our privacy statement _**should not**_ be seen by anyone but those directly working on this project.

In [0]:
"""
  1. Mount Google Drive (no need to check if already mounted, it does that for you)
  2. 
"""
from google.colab import drive
drive.mount('/content/gdrive')



In [0]:
!pip3 install torch
!pip3 install torchvision
!pip3 install tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
import gc
from IPython.core.ultratb import AutoFormattedTB
from torch.utils.data.sampler import SubsetRandomSampler

__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

assert torch.cuda.is_available(), "You need to request a GPU from Runtime > Change Runtime"

In [0]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


In [0]:
# Initialize Model
model = torchvision.models.resnet50(pretrained=True)
num_f = model.fc.in_features
model.fc = nn.Linear(num_f, len(dataset.classes))

model = model.cuda() #use GPU

In [0]:
model_file = "/content/gdrive/My Drive/CS474 Final Project/CustomFingerspelling/Models3/resnet50.mod"
model.load_state_dict(torch.load(model_file)["state"])

In [0]:
input_img = torch.randn(1, 3, 224, 224, requires_grad=True)

# Initialize Objective and Optimizer and other parameters
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam([input_img.requires_grad_()], lr=0.005)

In [0]:
# This is what was talked about in the video for memory management
num_epochs = 20
show_every = 5
train_losses = []

def scope():
  try:
    #your code for calling dataset and dataloader
    gc.collect()
    torch.cuda.empty_cache()

    loop = tqdm(num_epochs, position = 0)

    for epoch in range(num_epochs):
      optimizer.zero_grad()
      y_hat = model(input_img)
      y_long = torch.tensor([6]) # the letter g

      # print("\n\n-----------\ny_hat: {}, y_long: {}\n-----------\n\n".format(y_hat.size(), y_long.size()))
      loss = objective(y_hat, y_long)

      loop.set_description("epoch:{}, loss:{:.4f}".format(epoch, loss.item(), accuracy))
      loop.update(1)
      loss.backward()
      optimizer.step()
    
    if epoch % valid_frequency == 0:
      imshow(input_img)
  except:
    __ITB__()
    raise

In [0]:
scope()

epoch:43, loss:1.9621, accuracy:0.6000:  20%|█▉        | 993/5000 [07:36<31:06,  2.15it/s]

In [0]:
print(train_losses)
print(validation_losses)

In [0]:
plt.title("Losses")
plt.plot(*zip(*train_losses), label="Training")
plt.plot(*zip(*validation_losses), label="Validation")
plt.legend(loc="upper right")
plt.show()

plt.plot(*zip(*train_acc), label="Training")
plt.plot(*zip(*valid_acc), label="Validation")
plt.title("Training accuracies")
plt.show()

In [0]:
# Save that model out.
state = {
    'state': model.state_dict(),
    'optim': optimizer.state_dict(),
    'train_losses': train_losses,
    'train_acc': train_acc,
    'valid_losses': validation_losses,
    'valid_acc': valid_acc,
}
model_file = "/content/gdrive/My Drive/CS474 Final Project/CustomFingerspelling/Models3/resnet50.mod"
torch.save(state, model_file)